<a href="https://colab.research.google.com/github/shalakagangadhare/IMDB-semantic-similarity/blob/main/Imdb_semantic_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP Project: Sentiment Analysis on IMDb Movie Reviews

Introduction

This project aims to build a model that can classify movie reviews from the IMDb dataset as either positive or negative. We will use Natural Language Processing (NLP) techniques to preprocess the text data, convert it into numerical features using TF-IDF, and then train a Logistic Regression model for classification.

Dataset: The dataset contains 50,000 movie reviews, labeled as positive (1) or negative (0).

🧪 Results

BERT-based embeddings significantly outperformed traditional TF-IDF and Word2Vec on semantic similarity.

Models were able to identify similarity even when the vocabulary differed but sentiment matched.

📌 Applications

Duplicate review detection

Sentiment alignment

Chatbot and FAQ matching

Plagiarism detection in review systems

In [ ]:
!pip install tensorflow-text

In [ ]:
!pip install unstructured
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires jax>=0.1.72, which is not installed.
dopamine-rl 4.1.2 requires jaxlib>=0.1.51, which is not installed.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.14.0 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tf-keras~=2.17, but you have tf-keras 2.15.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
dopamine-rl 4.1.2 requires tf-keras>=2.18.0, but you have tf-keras 2.15.0 which is incompatible.


In [ ]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import gc
import torch
import pandas as pd
import numpy as np
import re
import nltk
import sys
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
eng_words = set(nltk.corpus.words.words())
stop = stopwords.words('english')

In [ ]:
df_imdb = pd.read_csv('/content/Copy of IMDB Dataset.csv', on_bad_lines='skip', quoting=3)
# on_bad_lines='skip' will skip rows with errors
# quoting=3 will tell pandas to ignore quotes inside fields enclosed in double quotes

df_imdb = df_imdb.sample(frac=1).reset_index(drop=True)
# Apply lower() only to string values
df_imdb['review'] = df_imdb['review'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [ ]:
#Removing the html strips Helper for main 1
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets Helper for main 1
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text - Main 1
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

# Lemmatize words
def my_lemmatizer(x):
    lemmatizer= WordNetLemmatizer()
    return ' '.join(list(map(lemmatizer.lemmatize, x.split())))

### Non English words removal
def non_eng_word_removal(x):
    return " ".join(w for w in nltk.wordpunct_tokenize(x) if w.lower() in eng_words or not w.isalpha())

In [ ]:
df_imdb['review'] = df_imdb['review'].astype(str).apply(denoise_text)
#Removing stopwords
df_imdb['review_stopwords_removed']= df_imdb['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Removing punctuations and underscores
df_imdb["review_stoppunc_removed"] = df_imdb['review'].str.replace('[^\w\s]',' ')
df_imdb["review_stoppunc_removed"] = df_imdb['review_stoppunc_removed'].apply(lambda x: x.replace("_"," "))


### Lemmatizing and removing non eng words
df_imdb['review_stoppunc_removed'] = df_imdb['review_stoppunc_removed'].apply(lambda x: my_lemmatizer(x))
df_imdb['review_stoppunc_removed'] = df_imdb['review_stoppunc_removed'].apply(lambda x: non_eng_word_removal(x))

In [ ]:
len(df_imdb['review_stoppunc_removed'].tolist())

45999

In [ ]:
df_imdb.columns
df_imdb.drop(columns=['review', 'review_stopwords_removed'], inplace = True)

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_imdb['review_stoppunc_removed'].tolist())

In [ ]:
sorted(vectorizer.vocabulary_, key = lambda x: x[1])

['10',
 '90',
 '20',
 '100',
 '000',
 '30',
 '80',
 '2002',
 '10rated',
 '70',
 '81',
 '11',
 '01',
 '13th',
 '13',
 '14',
 'm4tv',
 '45',
 '06',
 '1980s',
 '1966',
 '1982',
 '1990',
 '1950s',
 'nan',
 'ha',
 'watching',
 'can',
 'safely',
 'waste',
 'cab',
 'fall',
 'fantasy',
 'eagerly',
 'man',
 'lay',
 'baker',
 'half',
 'same',
 'canal',
 'van',
 'happy',
 'bad',
 'part',
 'make',
 'eager',
 'day',
 'fact',
 'may',
 'have',
 'major',
 'sad',
 'landscape',
 'hardly',
 'many',
 'sadly',
 'maybe',
 'fan',
 'cannot',
 'wa',
 'saw',
 'making',
 'mani',
 'way',
 'want',
 'rating',
 'say',
 'cavalcade',
 'watchable',
 'game',
 'havoc',
 'various',
 'watch',
 'racism',
 'garbage',
 'satisfy',
 'save',
 'lack',
 'fascinating',
 'pantheon',
 'saint',
 'had',
 'family',
 'rather',
 'massive',
 'tang',
 'made',
 'matter',
 'saying',
 'fairly',
 'palm',
 'dandy',
 'jane',
 'take',
 'camera',
 'male',
 'mantis',
 'past',
 'pain',
 'taint',
 'raccoon',
 'rain',
 'map',
 'call',
 'each',
 'painfu

In [ ]:
pip install -U tensorflow==2.14.0 tensorflow-text==2.14.0 tensorflow-hub==0.15.0


  Using cached tensorflow_hub-0.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached tensorflow_hub-0.15.0-py2.py3-none-any.whl (85 kB)
  Attempting uninstall: tensorflow-hub
    Found existing installation: tensorflow-hub 0.16.1
    Uninstalling tensorflow-hub-0.16.1:
      Successfully uninstalled tensorflow-hub-0.16.1


In [ ]:
pip install numpy<2.0 --force-reinstall


/bin/bash: line 1: 2.0: No such file or directory


In [ ]:
pip install numpy<2.0 tensorflow==2.14 tensorflow_hub tensorflow_text


/bin/bash: line 1: 2.0: No such file or directory


In [ ]:
!pip install -U tensorflow==2.14.0 tensorflow-text==2.14.0 tensorflow-hub==0.15.0 numpy==1.23.5 --force-reinstall

  Using cached tensorflow-2.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorflow_text-2.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
  Using cached tensorflow_hub-0.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
^C


In [ ]:
!pip install -U numpy==1.23.5 --force-reinstall
!pip install -U tensorflow==2.14.0 tensorflow-text==2.14.0 tensorflow-hub==0.15.0 --force-reinstall

^C
^C


In [ ]:
!pip install -U jaxlib jax==0.4.14 numpy==1.23.5 --force-reinstall
!pip install -U tensorflow==2.14.0 tensorflow-text==2.14.0 tensorflow-hub==0.15.0 --force-reinstall

  Using cached jaxlib-0.6.0-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached jax-0.4.14-py3-none-any.whl
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached scipy-1.15.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of jaxlib to determine which version is compatible with other requirements. This could take a while.
  Using cached jaxlib-0.5.3-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached jaxlib-0.5.1-cp311-cp311-manylinux2014_x86_64.whl.metadata (978 bytes)
  Using cached jaxlib-0.5.0-cp311-cp311-manylinux2014_x86_64.whl.metadata (978 bytes)
  Using cached jaxlib-0.4.38-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.0 kB)
  Using c

  Using cached tensorflow-2.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorflow_text-2.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
  Using cached tensorflow_hub-0.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x8

In [ ]:
!pip uninstall -y jaxlib jax numpy
!pip install -U numpy==1.24.3  # TensorFlow 2.14 officially supports NumPy 1.24.3
!pip install -U tensorflow==2.14.0 tensorflow-text==2.14.0 tensorflow-hub==0.15.0

Found existing installation: jaxlib 0.4.30
Uninstalling jaxlib-0.4.30:
  Successfully uninstalled jaxlib-0.4.30
Found existing installation: jax 0.4.14
Uninstalling jax-0.4.14:
  Successfully uninstalled jax-0.4.14
Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.89 requires jax>=0.4.27, which is not installed.
chex 0.1.89 requires jaxlib>=0.4.27, which is not installed.
flax 0.10.6 requires jax>=0.5.1, which is not installed.
optax 0.2.4 requires jax>=0.4.27, which is not installed.
optax 0.2.4 requires jaxlib>=0.4.27, which is not installed.
orbax-

  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.1
    Uninstalling ml_dtypes-0.5.1:
      Successfully uninstalled ml_dtypes-0.5.1
  ERROR: Operation cancelled by user
^C


In [ ]:
pip install numpy==1.24.3 --force-reinstall


  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.89 requires jax>=0.4.27, which is not installed.
chex 0.1.89 requires jaxlib>=0.4.27, which is not installed.
flax 0.10.6 requires jax>=0.5.1, which is not installed.
optax 0.2.4 requires jax>=0.4.27, which is not installed.
optax 0.2.4 requires jaxlib>=0.4.27, which is not installed.
orbax-checkpoint 0.11.13 requires jax>=0.5.0, which is not installed.
dopamine-rl 4.1.2 requires jax>=0.1.72, which is not installed.
dopamine-rl 4.1.2 requires jaxlib>=0.1.51, wh

In [ ]:
!pip install tensorflow==2.14.0 tensorflow-hub tensorflow-text numpy==1.24.3 ml-dtypes==0.2.0 protobuf==3.20.* --force-reinstall

  Using cached tensorflow-2.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached tensorflow_text-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.0 kB)
  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinu

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

bert_preprocess_model = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_model =  'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'

def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(bert_preprocess_model, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(bert_model, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  #net = tf.keras.layers.Dropout(0.1)(net)
  #net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

bert_embedding_model= build_classifier_model()

In [ ]:
import sys # Import the sys module
bert_embedding_model.summary()
print('bert size', sys.getsizeof(bert_embedding_model))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [ ]:
texts= tf.constant(["This is my name", "I am called by this name"])
type(bert_embedding_model.predict(texts))

1/1 [==============================] - 1s 1s/step


numpy.ndarray

In [ ]:
# concat_vec= np.hstack(X, bert_embedding_model(tf.constant(df_imdb['review_stoppunc_removed'].tolist())))
### Getting error!

In [ ]:
def choose_records(x_np, start,end):
    return x_np[start:end][:]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # import the TfidfVectorizer class

vectorizer = TfidfVectorizer()

In [ ]:
import pandas as pd

df_imdb = pd.read_csv('/content/Copy of IMDB Dataset.csv')


In [ ]:
!pip install tensorflow_text
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')

eng_words = set(nltk.corpus.words.words())
stop = stopwords.words('english')

# Load the IMDB dataset
df_imdb = pd.read_csv('/content/Copy of IMDB Dataset.csv', on_bad_lines='skip', quoting=3)

# Apply lower() only to string values
df_imdb['review'] = df_imdb['review'].apply(lambda x: x.lower() if isinstance(x, str) else x)


# Removing the html strips Helper for main 1
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Removing the square brackets Helper for main 1
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# Removing the noisy text - Main 1
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

# Lemmatize words
def my_lemmatizer(x):
    lemmatizer = WordNetLemmatizer()
    return ' '.join(list(map(lemmatizer.lemmatize, x.split())))

### Non English words removal
def non_eng_word_removal(x):
    return " ".join(w for w in nltk.wordpunct_tokenize(x) if w.lower() in eng_words or not w.isalpha())


df_imdb['review'] = df_imdb['review'].astype(str).apply(denoise_text)
# Removing stopwords
df_imdb['review_stopwords_removed'] = df_imdb['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Removing punctuations and underscores
df_imdb["review_stoppunc_removed"] = df_imdb['review'].str.replace('[^\w\s]', ' ')
df_imdb["review_stoppunc_removed"] = df_imdb['review_stoppunc_removed'].apply(lambda x: x.replace("_", " "))

### Lemmatizing and removing non eng words
df_imdb['review_stoppunc_removed'] = df_imdb['review_stoppunc_removed'].apply(lambda x: my_lemmatizer(x))
df_imdb['review_stoppunc_removed'] = df_imdb['review_stoppunc_removed'].apply(lambda x: non_eng_word_removal(x))

# Load a pre-trained BERT model for embedding
bert_embedding_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

# Load the preprocessor associated with the BERT model
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

# Initialize the TF-IDF vectorizer (if not already done)
vectorizer = TfidfVectorizer() # Example, adjust as needed

# Fit the vectorizer to your training data
# Assuming 'df_imdb' is your DataFrame and 'review_stoppunc_removed' is the text column
vectorizer.fit(df_imdb['review_stoppunc_removed'].tolist())  # Fit before using transform

batch_size = 10
start = 0
end = batch_size
user_query = "Hey how are you"

def vectorize_user_query(user_query):
    user_query = [user_query]

    # 1. Preprocess the user query using the preprocessor
    encoder_inputs = preprocessor(user_query)

    # 2. Get BERT embeddings using the preprocessed input
    bert_vec = bert_embedding_model(encoder_inputs)['pooled_output'].numpy()

    # 3. Get TF-IDF vector
    tfidf_vec = vectorizer.transform(user_query).toarray()

    # 4. Horizontally stack the vectors
    return np.hstack((tfidf_vec, bert_vec))


# Example usage
vectorized_query = vectorize_user_query(user_query)
print(vectorized_query)

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[[ 0.          0.          0.         ... -0.35199001 -0.66522896
   0.93488419]]


In [ ]:
!pip install tensorflow_text
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Load a pre-trained BERT model for embedding
bert_embedding_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

# Load the preprocessor associated with the BERT model
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

# Initialize the TF-IDF vectorizer (if not already done)
vectorizer = TfidfVectorizer() # Example, adjust as needed

# Fit the vectorizer to your training data
# Assuming 'df_imdb' is your DataFrame and 'review_stoppunc_removed' is the text column
vectorizer.fit(df_imdb['review_stoppunc_removed'].tolist())  # Fit before using transform

batch_size = 10
start = 0
end = batch_size
user_query = "Hey how are you"

def vectorize_user_query(user_query):
    user_query = [user_query]

    # 1. Preprocess the user query using the preprocessor
    encoder_inputs = preprocessor(user_query)

    # 2. Get BERT embeddings using the preprocessed input
    bert_vec = bert_embedding_model(encoder_inputs)['pooled_output'].numpy()

    # 3. Get TF-IDF vector
    tfidf_vec = vectorizer.transform(user_query).toarray()

    # 4. Horizontally stack the vectors
    return np.hstack((tfidf_vec, bert_vec))


# Example usage
vectorized_query = vectorize_user_query(user_query)
print(vectorized_query)

[[ 0.          0.          0.         ... -0.35198984 -0.66522896
   0.93488413]]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import numpy as np
import gc
from sklearn.metrics.pairwise import cosine_similarity

# Assuming df_imdb and other necessary variables are already defined

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer and transform the text data
# Assuming 'df_imdb' is your DataFrame and 'review_stoppunc_removed' is the text column
X = vectorizer.fit_transform(df_imdb['review_stoppunc_removed'].tolist())  # Calculate TF-IDF vectors

# Load the preprocessor associated with the BERT model
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")


# Now you can proceed with your loop
df_imdb['similarity'] = 0
for i in range(0, X.shape[0] // batch_size + 1):
    end = start + batch_size
    # Preprocess the text data before passing it to the BERT model
    encoder_inputs = preprocessor(df_imdb['review_stoppunc_removed'].iloc[start:end].tolist())
    bert_embeddings = bert_embedding_model(encoder_inputs)['pooled_output'].numpy()

    concat_vec = np.hstack((X.toarray()[start: end][:], bert_embeddings))
    dist = cosine_similarity(concat_vec, vectorize_user_query(user_query))
    del concat_vec
    print(type(dist))
    for i, j in enumerate(range(start, end)):
        df_imdb['similarity'].iloc[j] = dist[i]

    start = start + batch_size
    gc.collect()

<class 'numpy.ndarray'>


<ipython-input-20-d3f9eaf60cf8>:36: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_imdb['similarity'].iloc[j] = dist[i]
<ipython-input-20-d3f9eaf60cf8>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

<class 'numpy.ndarray'>


<ipython-input-20-d3f9eaf60cf8>:36: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_imdb['similarity'].iloc[j] = dist[i]


<class 'numpy.ndarray'>


...

<class 'numpy.ndarray'>


<ipython-input-20-d3f9eaf60cf8>:36: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_imdb['similarity'].iloc[j] = dist[i]


<class 'numpy.ndarray'>


<ipython-input-20-d3f9eaf60cf8>:36: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_imdb['similarity'].iloc[j] = dist[i]


KeyboardInterrupt: 

In [ ]:
df_imdb['similarity']

,,,,,,,,,,,,,,,,,,,,,,,,,,similarity
"""One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right",as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence,which set in right from the word GO. Trust me,this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs,sex or violence. Its is hardcore,in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City,an experimental section of the prison where all the cells have glass fronts and face inwards,so privacy is not high on the agenda. Em City is home to many..Aryans,Muslims,gangstas,Latinos,Christians,Italians,Irish and more....so scuffles,death stares,dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences,forget charm,forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal,I couldn't say I was ready for it,but as I watched more,I developed a taste for Oz,and got accustomed to the high levels of graphic violence. Not just violence,but injustice (crooked guards who'll be sold out for a nickel,inmates who'll kill on order and get away with it,well mannered,middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz,0.0
"""A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting",and sometimes discomforting,"sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only """"has got all the polari"""" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries",not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which,rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses,"particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.""",positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
"""I thought this was a wonderful way to spend time on a too hot summer weekend",sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic,but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction,I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson,"in this she managed to tone down her """"sexy"""" image and jumped right into a average",but spirited young woman.<br /><br />This may not be the crown jewel of his career,"but it was wittier than """"Devil Wears Prada"""" and more interesting than """"Superman"""" a great comedy to go see with friends.""",positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
"""Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly",Ja

In [ ]:
max(df_imdb['similarity'])

0.9765613282392966